<a href="https://colab.research.google.com/github/fatimazain118/Deep-Learning-Assignments/blob/main/krishNaik/tut27_OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner  #used to optimised our CNN model also how many layer how many dense layer you have to used to get the output

     |████████████████████████████████| 98 kB 3.5 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__) #as it's the prerequisite to use keras tuner

2.7.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels),(test_images,test_labels) = fashion_mnist.load_data()  #dividing the data into training and testing data after loading it 

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0  #scaling down the data so that it can comes down between 0 or 1
test_images = test_images/255.0

In [7]:
train_images

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [8]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [9]:
train_images[0].shape  #it,s 28*28 pixels

(28, 28)

In [10]:
train_images=train_images.reshape(len(train_images), 28,28,1)
test_images=test_images.reshape(len(test_images), 28,28,1)    #reshapping into 4 dimensions

1st parameter we have to pass is filter, and with the help of the keras tuner we can select different different values for this filter, hp.Int will create the range of integer with name conv_1_filter having min filters 32 and max filter 128. With size 3 or 5

In [11]:
def build_model(hp):       #hp is hyper-parameter
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),   #filter size
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),  #no. of neurons 32-128
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',    #normally used in classification pblm in cnn
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch  #this randomSearch will tell which parameter will be best for this model
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [13]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials = 5,
                            directory = 'output',
                            project_name = "Mnist Fashion")

In [14]:
tuner_search.search(train_images, train_labels, epochs=3,validation_split = 0.1)

Trial 5 Complete [00h 01m 23s]
val_accuracy: 0.9131666421890259

Best val_accuracy So Far: 0.9166666865348816
Total elapsed time: 00h 06m 23s
INFO:tensorflow:Oracle triggered exit


In [18]:
model = tuner_search.get_best_models(num_models=1)[0]  #gives as a best model so far

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        27712     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 80)                2949200   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 2,978,202
Trainable params: 2,978,202
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.1141 - accuracy: 0.9578 - val_loss: 0.2480 - val_accuracy: 0.9182
Epoch 5/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0770 - accuracy: 0.9726 - val_loss: 0.3057 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0528 - accuracy: 0.9805 - val_loss: 0.3569 - val_accuracy: 0.9093
Epoch 7/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0375 - accuracy: 0.9870 - val_loss: 0.4001 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0300 - accuracy: 0.9894 - val_loss: 0.4224 - val_accuracy: 0.9113
Epoch 9/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0233 - accuracy: 0.9914 - val_loss: 0.4961 - val_accuracy: 0.9150
Epoch 10/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0180 - accuracy: 0.9938 - val_loss: 0.5378 - va